In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import io
import requests

# Declare World's ISO Alpha-3 Code value to WRD
world_iso3 = 'WRD'


### Countries to observe

In [2]:
# Load Countries to observe into DataFrame
country_csv_path = "Resources/Country/Country.csv"
country_df = pd.read_csv(country_csv_path)
# Set World's ISO Alpha-3 Code value to WRD
country_df = country_df.set_index('Country Name')
country_df.loc['World', 'ISO alpha-3 Code'] = world_iso3
country_df = country_df.reset_index()
country_df.head()

,Country Name,GFN Country Code,ISO alpha-2 Code,ISO alpha-3 Code
0,United States of America,231,US,USA
1,Australia,10,AU,AUS
2,Brazil,21,BR,BRA
3,Canada,33,CA,CAN
4,China,351,CN,CHN


### Nation Footprint Data

In [3]:
# Load National Footprint Data
nf_csv_path = "Resources/NFA/NFA 2018.csv"
nf_df = pd.read_csv(nf_csv_path)

# Set World's ISO Alpha-3 Code value to WRD
nf_df = nf_df.set_index('country')
nf_df.loc['World', 'ISO alpha-3 code'] = world_iso3
nf_df = nf_df.reset_index()

# Filters: 
# records keeping: BiocapTotGHA 
# records filter out: BiocapPerCap, EFConsPerCap/TotGHA, EFExportsPerCap/TotGHA, EFImportsPerCap/TotGHA, EFProdPerCap/TotGHA
nf_df = nf_df[nf_df['record'] == 'BiocapTotGHA']
nf_df.head()

,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
1,Armenia,ARM,Asia,Western Asia,1992,BiocapTotGHA,555812.9726,465763.3374,289190.6623,47320.22459,116139.5982,0.0,1474226.795,949.033,3449000
11,Armenia,ARM,Asia,Western Asia,1993,BiocapTotGHA,538659.0959,466181.5956,289695.8367,47293.87727,107720.2388,0.0,1449550.644,886.033,3370000
21,Armenia,ARM,Asia,Western Asia,1994,BiocapTotGHA,537897.6656,473441.0226,289832.8036,47206.96501,103552.5070,0.0,1451930.964,956.471,3290000
31,Armenia,ARM,Asia,Western Asia,1995,BiocapTotGHA,531812.0787,505784.7756,289527.2356,47069.84016,101521.3116,0.0,1475715.242,1043.540,3223000
41,Armenia,ARM,Asia,Western Asia,1996,BiocapTotGHA,612400.5242,538818.3582,289980.1739,47010.04117,112371.3847,0.0,1600580.482,1121.880,3173000


In [4]:
# Merge National Footprint Data with Countries to observe
merged_nf_df = pd.merge(country_df, nf_df, how='left', left_on='ISO alpha-3 Code', right_on='ISO alpha-3 code')
merged_nf_df.head()

,Country Name,GFN Country Code,ISO alpha-2 Code,ISO alpha-3 Code,country,ISO alpha-3 code,UN_region,UN_subregion,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,Percapita GDP (2010 USD),population
0,United States of America,231,US,USA,United States of America,USA,North America,North America,1961,BiocapTotGHA,224333466.7,99391016.28,513161544.7,102673009.1,7119856.665,0.0,946678893.4,17142.2,189077000
1,United States of America,231,US,USA,United States of America,USA,North America,North America,1962,BiocapTotGHA,218025900.3,99015723.73,512893594.4,102432709.8,7159264.888,0.0,939527193.0,17910.3,191861000
2,United States of America,231,US,USA,United States of America,USA,North America,North America,1963,BiocapTotGHA,230020245.0,97053163.13,511938209.3,102086380.0,7554383.049,0.0,948652380.5,18431.2,194514000
3,United States of America,231,US,USA,United States of America,USA,North America,North America,1964,BiocapTotGHA,216878024.0,96494835.45,511382325.4,101866885.8,7279098.246,0.0,933901168.8,19231.2,197029000
4,United States of America,231,US,USA,United States of America,USA,North America,North America,1965,BiocapTotGHA,240650193.3,95365422.79,511433207.6,101503424.2,8218387.830,0.0,957170635.8,20207.8,199404000


### Food and Agriculture Organization of the UN Data

In [5]:
# Set FAO stardand url parametrs
fao_area = 'area=' + '%2C'.join(country_df['GFN Country Code'].apply(str))
fao_year = '&year=' + '%2C'.join(np.arange(1995,2019).astype(str))
fao_area_cs = '&area_cs=FAO'
fao_item_cs = '&item_cs=FAO'
fao_tail = '&show_codes=true&show_unit=true&show_flags=true&null_values=false&output_type=csv'

### Production: Livestock Primary

In [6]:
# Production: Livestock Primary
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/QL?'
element_str = '2313%2C2510'
fao_element = '&element=' + element_str
item_str = '1765'
fao_item = '&item=' + item_str
# finalize fao url
pr_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Get Production: Livestock Primary Data from FOA
response = requests.get(pr_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_pr_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_pr_df.head()


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QL,Livestock Primary,10,Australia,5510,Production,1765,"Meat, Total",1995,1995,tonnes,3297625,A,"Aggregate, may include official, semi-official..."
1,QL,Livestock Primary,10,Australia,5510,Production,1765,"Meat, Total",1996,1996,tonnes,3192245,A,"Aggregate, may include official, semi-official..."
2,QL,Livestock Primary,10,Australia,5510,Production,1765,"Meat, Total",1997,1997,tonnes,3340594,A,"Aggregate, may include official, semi-official..."
3,QL,Livestock Primary,10,Australia,5510,Production,1765,"Meat, Total",1998,1998,tonnes,3601278,A,"Aggregate, may include official, semi-official..."
4,QL,Livestock Primary,10,Australia,5510,Production,1765,"Meat, Total",1999,1999,tonnes,3638049,A,"Aggregate, may include official, semi-official..."


### Trade : Livestock Products

In [7]:
# Trade: Livestock Products (Import/Export)
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/TP?'
element_str = '2910%2C2610'
fao_element = '&element=' + element_str
item_str = '2077'
fao_item = '&item=' + item_str
# finalize fao url
tr_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Trade: Livestock Products Data from FOA
response = requests.get(tr_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_tr_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_tr_df.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,TP,Crops and livestock products,10,Australia,5610,Import Quantity,2077,Total Meat,1995,1995,tonnes,13508,A,"Aggregate, may include official, semi-official..."
1,TP,Crops and livestock products,10,Australia,5610,Import Quantity,2077,Total Meat,1996,1996,tonnes,16622,A,"Aggregate, may include official, semi-official..."
2,TP,Crops and livestock products,10,Australia,5610,Import Quantity,2077,Total Meat,1997,1997,tonnes,19469,A,"Aggregate, may include official, semi-official..."
3,TP,Crops and livestock products,10,Australia,5610,Import Quantity,2077,Total Meat,1998,1998,tonnes,14334,A,"Aggregate, may include official, semi-official..."
4,TP,Crops and livestock products,10,Australia,5610,Import Quantity,2077,Total Meat,1999,1999,tonnes,32362,A,"Aggregate, may include official, semi-official..."


### Food Balance: Commodity Balances Livestocks

In [8]:
# Food Balance: Commodity Balances Livestocks (Domestic Supply Qty)
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/BL?'
element_str = '2300'
fao_element = '&element=' + element_str
item_str = '2943'
fao_item = '&item=' + item_str
# finalize fao url
cb_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Get Food Balance: Commodity Balances Livestocks Data from FOA
response = requests.get(cb_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_cb_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_cb_df.head()


,Domain Code,Domain,Country Code,Country,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,BL,Commodity Balances - Livestock and Fish Primar...,10,Australia,5300,Domestic supply quantity,2943,Meat,1995,1995,tonnes,2035358,A,"Aggregate, may include official, semi-official..."
1,BL,Commodity Balances - Livestock and Fish Primar...,10,Australia,5300,Domestic supply quantity,2943,Meat,1996,1996,tonnes,2042133,A,"Aggregate, may include official, semi-official..."
2,BL,Commodity Balances - Livestock and Fish Primar...,10,Australia,5300,Domestic supply quantity,2943,Meat,1997,1997,tonnes,1989340,A,"Aggregate, may include official, semi-official..."
3,BL,Commodity Balances - Livestock and Fish Primar...,10,Australia,5300,Domestic supply quantity,2943,Meat,1998,1998,tonnes,2134020,A,"Aggregate, may include official, semi-official..."
4,BL,Commodity Balances - Livestock and Fish Primar...,10,Australia,5300,Domestic supply quantity,2943,Meat,1999,1999,tonnes,2163870,A,"Aggregate, may include official, semi-official..."


### Investment: Government Expenditure

In [9]:
# Investment: Government Expenditure (Subsidies)
# Set url parametrs
fao_base_url = 'http://fenixservices.fao.org/faostat/api/v1/en/data/IG?'
element_str = '6110'
fao_element = '&element=' + element_str
item_str = '23164%2C23134'
fao_item = '&item=' + item_str
# finalize fao url
fb_fao_url = fao_base_url + fao_area + fao_area_cs + fao_element + fao_item  + fao_item_cs + fao_year + fao_tail

# Request Get Investment: Government Expenditure Data from FOA
response = requests.get(fb_fao_url)
decoded_content = response.content.decode('ISO-8859-1')
# Store CSV to DataFrame
fao_fb_df = pd.read_csv(io.BytesIO(response.content), encoding='ISO-8859-1')
fao_fb_df.head()


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,IG,Government Expenditure,21,Brazil,6110,Value US$,23134,Agriculture (General Government),2009,2009,millions,4508.80,Qm,Official data from questionnaires and/or natio...,NaN
1,IG,Government Expenditure,21,Brazil,6110,Value US$,23134,Agriculture (General Government),2010,2010,millions,4715.52,Qm,Official data from questionnaires and/or natio...,NaN
2,IG,Government Expenditure,21,Brazil,6110,Value US$,23134,Agriculture (General Government),2011,2011,millions,5667.72,Qm,Official data from questionnaires and/or natio...,NaN
3,IG,Government Expenditure,21,Brazil,6110,Value US$,23134,Agriculture (General Government),2012,2012,millions,5815.74,Qm,Official data from questionnaires and/or natio...,NaN
4,IG,Government Expenditure,21,Brazil,6110,Value US$,23134,Agriculture (General Government),2013,2013,millions,5968.06,Qm,Official data from questionnaires and/or natio...,NaN


### OECD: Meat Consumption

In [10]:
# Load FAO Meat Consumption Data (downloaded from OECD)
fao_mc_csv_path = "Resources/FAO/daily-protein-supply-from-animal-and-plant-based-foods.csv"
fao_mc_df = pd.read_csv(fao_mc_csv_path)

merged_fao_mc_df = pd.merge(country_df, fao_mc_df, how='left', left_on='ISO alpha-3 Code', right_on='Code')
merged_fao_mc_df.head()

,Country Name,GFN Country Code,ISO alpha-2 Code,ISO alpha-3 Code,Entity,Code,Year,Daily protein supply of animal origin (g/person/day),Daily protein supply of plant origin (g/person/day)
0,United States of America,231,US,USA,United States,USA,1947.0,63.00,36.20
1,United States of America,231,US,USA,United States,USA,1948.0,60.30,34.70
2,United States of America,231,US,USA,United States,USA,1961.0,62.99,32.22
3,United States of America,231,US,USA,United States,USA,1962.0,62.70,31.88
4,United States of America,231,US,USA,United States,USA,1963.0,63.70,31.90
